In [ ]:
!nvidia-smi 

Tue Oct 25 04:44:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Global-Local Image-based Active Contour Loss for Multiclass Medical Image Segmentation with Deep Learning/ISIC'
%run Unet_2D.ipynb
%run preprocessing_2D.ipynb
%run otherUnets.ipynb

In [3]:
!pip install pytorch-lightning
!pip install timm
import pytorch_lightning as pl
from IPython.display import clear_output
import nibabel as nib
import csv
import os
import glob 
import torch
import timm.optim
import torch.nn as nn
from timm.data.loader import create_loader
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
import pandas as pd
from fastprogress import master_bar, progress_bar
from torchvision import transforms
import torch.optim as optim
from itertools import product
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class HistoryLogger(pl.callbacks.Callback):
    def __init__(self, dir = "history_isic_AC.csv"):
        self.dir = dir
    def on_validation_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        if "loss_epoch" in metrics.keys():
            logs = {"epoch": trainer.current_epoch}
            keys = ["loss_epoch", "train_dice_epoch", "train_jac_epoch", "val_loss","val_dice", "val_jac"]
            for key in keys:
                logs[key] = metrics[key].item()
            header = list(logs.keys())
            isFile = os.path.isfile(self.dir)
            with open(self.dir, 'a', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=header)
                if not isFile:
                    writer.writeheader()
                writer.writerow(logs) 
        else:
            pass
def setDropProb(model, prob=0.01):
    for layer in model.modules():
        if isinstance(layer, DropBlock2D):
            layer.drop_prob = prob
clear_output()
############## turn off Debug APIs for Final Training############
torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

In [ ]:
#sun09
'''x_train = np.load('/content/drive/MyDrive/MRI_ACDC/Dataset2/Sunny_Brook_MRI/x_train_crop_192_256_epi_sun09.npy')
y_train = np.load('/content/drive/MyDrive/MRI_ACDC/Dataset2/Sunny_Brook_MRI/y_train_crop_192_256_epi_sun09.npy')
x_test = np.load('/content/drive/MyDrive/MRI_ACDC/Dataset2/Sunny_Brook_MRI/x_test_crop_192_256_epi_sun09.npy')
y_test = np.load('/content/drive/MyDrive/MRI_ACDC/Dataset2/Sunny_Brook_MRI/y_test_crop_192_256_epi_sun09.npy')'''

"x_train = np.load('/content/drive/MyDrive/MRI_ACDC/Dataset2/Sunny_Brook_MRI/x_train_crop_192_256_epi_sun09.npy')\ny_train = np.load('/content/drive/MyDrive/MRI_ACDC/Dataset2/Sunny_Brook_MRI/y_train_crop_192_256_epi_sun09.npy')\nx_test = np.load('/content/drive/MyDrive/MRI_ACDC/Dataset2/Sunny_Brook_MRI/x_test_crop_192_256_epi_sun09.npy')\ny_test = np.load('/content/drive/MyDrive/MRI_ACDC/Dataset2/Sunny_Brook_MRI/y_test_crop_192_256_epi_sun09.npy')"

In [ ]:
#ISIC17
'''data_train = np.load('/content/drive/MyDrive/MRI_ACDC/Dataset2/Sunny_Brook_MRI/train_crop_128_endo_sun09.npz')
x_train, y_train = data_train["image"], data_train["mask"]
data_test = np.load('/content/drive/MyDrive/MRI_ACDC/Dataset2/Sunny_Brook_MRI/test_crop_128_endo_sun09.npz')
x_test, y_test = data_test["image"], data_test["mask"]'''

'data_train = np.load(\'/content/drive/MyDrive/MRI_ACDC/Dataset2/Sunny_Brook_MRI/train_crop_128_endo_sun09.npz\')\nx_train, y_train = data_train["image"], data_train["mask"]\ndata_test = np.load(\'/content/drive/MyDrive/MRI_ACDC/Dataset2/Sunny_Brook_MRI/test_crop_128_endo_sun09.npz\')\nx_test, y_test = data_test["image"], data_test["mask"]'

In [4]:
#ISIC18
# data = np.load("./dataISIC2018/ISIC2018_192_256.npz")
# x, y = data["image"], data["mask"]
# test_size = int((20/100)*x.shape[0])
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=520, random_state=312)
# del x, y, data

In [22]:
#PH2
data = np.load('/content/drive/MyDrive/Global-Local Image-based Active Contour Loss for Multiclass Medical Image Segmentation with Deep Learning/ISIC/PH2Nhat/PH2_192_256.npz')
X_train, Y_train = data["image"], data["mask"]
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=30, random_state=312)

# len_dev = 30
# len_train = X_train.shape[0] - len_dev
# rand_index = np.random.permutation(X_train.shape[0], random_state=312)
# x_test = X_train[rand_index[:len_dev]]
# y_test = Y_train[rand_index[:len_dev]]
# x_train = X_train[rand_index[len_dev:]]
# y_train = Y_train[rand_index[len_dev:]]

In [23]:
columns = ['0','1']
df = pd.DataFrame(columns=columns)
_, counts =  np.unique(y_train, return_counts=True)
conts_dict = dict(zip(columns, counts))
df = df.append(conts_dict, ignore_index=True)
label_0 = df['0'].sum()
label_1 = df['1'].sum()
total_labels = label_0 + label_1
n_classes = 2
#Class weights claculation: n_samples / (n_classes * n_samples_for_class)
wt0 = round((total_labels/(n_classes*label_0)), 2)
wt1 = round((total_labels/(n_classes*label_1)), 2)
print("\n Weights are:", wt0, wt1)
# 0.63 2.37


 Weights are: 0.75 1.49


In [24]:
# train_dataset = DataLoader(ISICLoader(x_train, y_train), batch_size=8, pin_memory=True,
#                         shuffle=True, num_workers=2, 
#                         drop_last=True, prefetch_factor = 8)
# val_dataset = DataLoader(ISICLoader(x_val, y_val, typeData="test"), batch_size=16,
#                           num_workers=2, prefetch_factor=16)
# test_dataset = DataLoader(ISICLoader(x_test, y_test, typeData="test"), batch_size=16,
#                           num_workers=2, prefetch_factor=16)
# if torch.cuda.is_available():
#     train_dataset = create_loader(ISICLoader(x_train, y_train), batch_size=8, pin_memory=True,
#                         shuffle=True, num_workers=2, 
#                         drop_last=True, prefetch_factor = 8)
#     test_dataset = create_loader(ISICLoader(x_test, y_test, typeData="test"), batch_size=16,
#                           num_workers=2, prefetch_factor=16)
# else:
train_dataset = DataLoader(ISICLoader(x_train, y_train), batch_size=8, pin_memory=True, shuffle=True, num_workers=2, drop_last=True, prefetch_factor = 8)
test_dataset = DataLoader(ISICLoader(x_test, y_test, typeData="test"), batch_size=16, num_workers=2, prefetch_factor=16)

In [25]:
class SemiActiveLoss(nn.Module):
    def __init__(self, device, alpha =1e-9, beta = 1e-1, lamda = 1e-3):
        super().__init__()
        self.device = device
        self.alpha = alpha
        self.beta = beta      
        self.lamda = lamda
    def LevelsetLoss(self, image, y_pred, kernel_size=5, smooth=1e-5):
        kernel = torch.ones(1, y_pred.size(1), kernel_size, kernel_size, device=self.device) / kernel_size**2
        padding = kernel_size //2
        lossRegion = 0.0
        y_pred_fuzzy = y_pred ** 2
        for ich in range(image.size(1)):
            target_ = image[:,ich:ich+1] 
            pcentroid_local = F.conv2d(target_ * y_pred_fuzzy + smooth, kernel, padding = padding) \
                                / F.conv2d(y_pred_fuzzy + smooth, kernel, padding = padding)
            plevel_local = target_ - pcentroid_local
            loss_local = plevel_local * plevel_local * y_pred_fuzzy

            pcentroid_global = torch.sum(target_ * y_pred_fuzzy, dim=(2,3),keepdim=True) \
                                / torch.sum(y_pred_fuzzy+smooth, dim=(2,3),keepdim = True)   
            plevel_global = target_ - pcentroid_global
            loss_global = plevel_global * plevel_global * y_pred_fuzzy

            lossRegion += torch.sum(loss_local) + self.beta * torch.sum(loss_global)
        return lossRegion 
    def GradientLoss(self, y_pred, penalty = "l1"):
        dH = torch.abs(y_pred[...,1:] - y_pred[...,:-1])
        dW = torch.abs(y_pred[:,:,1:] - y_pred[:,:,:-1])
        if penalty == "l2":
            dH = dH * dH
            dW = dW * dW
        loss =  torch.sum(dH) +  torch.sum(dW)
        return loss
    def ActiveContourLoss(self, y_true, y_pred, smooth=1e-6):   
        dim = (1,2,3)
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)[:,1:]
        y_pred = y_pred[:,1:]

        active = - torch.log(1-y_pred+smooth) * (1-yTrueOnehot) - torch.log(y_pred+smooth) * yTrueOnehot
        loss = torch.sum(active, dim = dim) / torch.sum(yTrueOnehot + y_pred - yTrueOnehot * y_pred + smooth, dim = dim)
        return torch.mean(loss)

    def forward(self, image, y_true, y_pred):
        active = self.ActiveContourLoss(y_true, y_pred)
        # levelset =  self.LevelsetLoss(image, y_pred)
        # length = self.GradientLoss(y_pred)
        return active
        # return active + self.alpha * (levelset + self.lamda * length) 

In [26]:
class ActiveFocalLoss(nn.Module):
    def __init__(self, device, class_weight=[1] * NUM_CLASS, gamma=2):
        """
        class weight should be a list. 
        """
        super().__init__()
        self.device = device
        self.class_weight = torch.tensor(class_weight, device=device)
        self.gamma = gamma
    def forward(self, y_true, y_pred):
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)
        y_pred = torch.clamp(y_pred,min=1e-7,max=1-1e-7)

        active_focal = - yTrueOnehot * (1-y_pred)**self.gamma * torch.log(y_pred) \
                        - (1 - yTrueOnehot) * y_pred**self.gamma * torch.log(1 - y_pred)
        loss = torch.sum(active_focal, dim=[2, 3]) * self.class_weight
        return torch.sum(loss) / (torch.sum(self.class_weight) * y_true.size(0) * y_true.size(2) * y_true.size(3))

class ActiveContourLoss(nn.Module):
    def __init__(self, device, class_weight=[1] * NUM_CLASS):
        """
        class weight should be a list. 
        """
        super().__init__()
        self.device = device
        self.class_weight = torch.tensor(class_weight, device=device)
    def forward(self, y_true, y_pred):   
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)

        # y_pred = torch.clamp(y_pred,min=1e-7,max=1-1e-7)
        active = torch.sum(yTrueOnehot * (1 - y_pred) + (1 - yTrueOnehot) * y_pred, dim=[2, 3])
        loss = torch.sum(active * self.class_weight)
        return loss / (torch.sum(self.class_weight) * y_true.size(0) * y_true.size(2) * y_true.size(3))

In [27]:
class DiceLoss(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.device = device
    def forward(self, y_true, y_pred):   
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)[:, 1:]
        y_pred = y_pred[:,1:]

        intersection = torch.sum(yTrueOnehot * y_pred, dim=[1,2,3])
        cardinality  = torch.sum(yTrueOnehot + y_pred , dim=[1,2,3])
        loss = 1.0-torch.mean((2. * intersection + 1e-5) / (cardinality + 1e-5))
        return loss

In [28]:
class TverskyLoss(nn.Module):
    def __init__(self, device, alpha = 0.7):
        super().__init__()
        self.device = device
        self.alpha = alpha
    def forward(self, y_true, y_pred):   
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)[:, 1:]
        y_pred = y_pred[:,1:]

        TP = torch.sum(yTrueOnehot * y_pred, dim=[1,2,3])
        FN = torch.sum(yTrueOnehot * (1-y_pred), dim=[1,2,3])
        FP = torch.sum((1-yTrueOnehot) * y_pred, dim=[1,2,3])
        loss = 1-torch.mean((TP + 1e-5) / (TP + self.alpha*FN + (1-self.alpha)*FP + 1e-5))
        return loss

In [29]:
class Segmentor(pl.LightningModule):
    def __init__(self, model = PiDiNet(img_channel=3, inplane=32, dil=2, sa=True, ta=False)):
        super().__init__()
        self.model = model
    def forward(self, x):
        return self.model(x)
    def get_metrics(self):
        # don't show the version number
        items = super().get_metrics()
        items.pop("v_num", None)
        return items

    # def _step(self, batch):
    #     image, y_true = batch
    #     y_pred = self.model(image)
    #     loss = SemiActiveLoss(device=self.device)(image, y_true, y_pred)
    #     dice_score, jaccard_score = dice(y_true, y_pred), jaccard(y_true, y_pred)
    #     return loss, dice_score, jaccard_score    
    def _step(self, batch):
        image, y_true = batch
        y_pred = self.model(image)
        loss_focal = ActiveFocalLoss(self.device, [0.51, 22.79])(y_true, y_pred)
        loss_contour = ActiveContourLoss(self.device, [0.51, 22.79])(y_true, y_pred)
        loss = loss_focal + loss_contour
        dice_score, jaccard_score = dice(y_true, y_pred), jaccard(y_true, y_pred)
        return loss, dice_score, jaccard_score

    def training_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"loss": loss, "train_dice": dice_score, "train_jac": jaccard_score}
        self.log_dict(metrics, on_step=True, on_epoch=True, prog_bar = True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"val_loss": loss, "val_dice": dice_score, "val_jac":jaccard_score}
        self.log_dict(metrics, prog_bar = True)
        return metrics

    def test_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"test_loss": loss, "test_dice": dice_score, "test_jac":jaccard_score}
        self.log_dict(metrics, prog_bar = True)
        return metrics
        

    def configure_optimizers(self):
        optimizer = timm.optim.Nadam(self.parameters(), lr=1e-3)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor = 0.5, patience=16, verbose =True)
        lr_schedulers = {"scheduler": scheduler, "monitor": "val_dice"}
        return [optimizer], lr_schedulers
    

initialization done


In [31]:
segmentor = Segmentor(PiDiNet(img_channel=3, inplane=32, dil=2, sa=True, ta=False)) #Segmentor(U_Net(drop_prob=0)) 
os.makedirs('/content/weights/', exist_ok = True) 
check_point = pl.callbacks.model_checkpoint.ModelCheckpoint("/content/weights/", filename="ckpt{val_dice:0.4f}",
                                                            monitor="val_dice", mode = "max", save_top_k =1,
                                                            verbose=True, save_weights_only=True,
                                                            auto_insert_metric_name=False,)
progress_bar = pl.callbacks.TQDMProgressBar(refresh_rate=1)
logger = HistoryLogger()
#swa = pl.callbacks.StochasticWeightAveraging(swa_lrs=1e-3,swa_epoch_start=25)
PARAMS = {"gpus":1, "benchmark":True, "enable_progress_bar":True, "logger":False,
        #   "callbacks" : [progress_bar],
        #    "overfit_batches" :1,
          "callbacks" : [check_point, progress_bar, logger],
          "log_every_n_steps" :1, "num_sanity_val_steps":0, "max_epochs":2000,
          "precision":16,
          }

trainer = pl.Trainer(**PARAMS)
# segmentor = Segmentor.load_from_checkpoint(checkpoint_path="/content/drive/MyDrive/Global-Local Image-based Active Contour Loss for Multiclass Medical Image Segmentation with Deep Learning/ISIC/weights/propose.ckpt")
# segmentor = Segmentor.load_from_checkpoint(checkpoint_path="./weights/proposed+ta_0.9173_0.8566.ckpt")


initialization done


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(segmentor, train_dataset, test_dataset)

In [ ]:
trainer.save_checkpoint("/content/weights/ckpt0.8933.ckpt", weights_only=True)

In [ ]:
for layer in segmentor.modules():
    if isinstance(layer, DropBlock2D):
        layer.drop_prob = 0.05
        
for layer in segmentor.modules():
    if isinstance(layer, DropBlock2D):
        print(layer.drop_prob) 

# segmentor.configure_optimizers()

0.05
0.05


In [33]:
trainer.test(segmentor, test_dataset)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_dice            0.931552529335022
        test_jac            0.8777281641960144
        test_loss           0.3238128423690796
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.3238128423690796,
  'test_dice': 0.931552529335022,
  'test_jac': 0.8777281641960144}]

In [ ]:
# curn
class Segmentor(pl.LightningModule):
    def __init__(self, model = PiDiNet(img_channel=3, inplane=32, dil=2, sa=False, ta=True)):
        super().__init__()
        self.model = model
    def forward(self, x):
        return self.model(x)
    def get_metrics(self):
        # don't show the version number
        items = super().get_metrics()
        items.pop("v_num", None)
        return items

    # def _step(self, batch):
    #     image, y_true = batch
    #     y_pred = self.model(image)
    #     loss = SemiActiveLoss(device=self.device)(image, y_true, y_pred)
    #     dice_score, jaccard_score = dice(y_true, y_pred), jaccard(y_true, y_pred)
    #     return loss, dice_score, jaccard_score    
    def _step(self, batch):
        image, y_true = batch
        y_pred = self.model(image)
        loss_focal = ActiveFocalLoss(self.device, [0.51, 22.79])(y_true, y_pred)
        loss_contour = ActiveContourLoss(self.device, [0.51, 22.79])(y_true, y_pred)
        loss = loss_focal + loss_contour
        dice_score, jaccard_score = dice(y_true, y_pred), jaccard(y_true, y_pred)
        return loss, dice_score, jaccard_score

    def training_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"loss": loss, "train_dice": dice_score, "train_jac": jaccard_score}
        self.log_dict(metrics, on_step=True, on_epoch=True, prog_bar = True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"val_loss": loss, "val_dice": dice_score, "val_jac":jaccard_score}
        self.log_dict(metrics, prog_bar = True)
        return metrics

    def test_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"test_loss": loss, "test_dice": dice_score, "test_jac":jaccard_score}
        self.log_dict(metrics, prog_bar = True)
        return metrics
        

    def configure_optimizers(self):
        optimizer = timm.optim.Nadam(self.parameters(), lr=1e-3)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max",
                                                         factor = 0.5, patience=16, verbose =True)
        lr_schedulers = {"scheduler": scheduler, "monitor": "val_dice"}
        return [optimizer], lr_schedulers
    
# weight_path = sorted(glob.glob("./weights/*"), reverse=True)
# for weight in tqdm(weight_path):
#     print(weight)
segmentor = Segmentor(model = PiDiNet(img_channel=3, inplane=32, dil=8, sa=False, ta=True))
segmentor = Segmentor.load_from_checkpoint(checkpoint_path='./weights/proposed+ta_0.9525_0.9104.ckpt')
trainer.test(segmentor, test_dataset)
    #dice_score = trainer.test(segmentor, test_dataset)

In [ ]:
weight_path = sorted(glob.glob("/content/drive/MyDrive/Global-Local Image-based Active Contour Loss for Multiclass Medical Image Segmentation with Deep Learning/ISIC/weights/*"), reverse=True)
max_score = 0
best_weight = ""
for weight in tqdm(weight_path):
    segmentor = Segmentor.load_from_checkpoint('/content/drive/MyDrive/Global-Local Image-based Active Contour Loss for Multiclass Medical Image Segmentation with Deep Learning/ISIC/weights/ckpt0.9141.ckpt') #checkpoint_path=weight
    dice_score = trainer.test(segmentor, test_dataset)[0]["test_dice"]
    clear_output()
    print(dice_score)
    if max_score < dice_score:
        max_score = dice_score
        best_weight = weight
print(f"best weight is: {best_weight} with {max_score}")


100%|██████████| 15/15 [00:43<00:00,  2.90s/it]

0.9038322567939758
best weight is: /content/drive/MyDrive/Global-Local Image-based Active Contour Loss for Multiclass Medical Image Segmentation with Deep Learning/ISIC/weights/proposed+ta_0.9525_0.9104.ckpt with 0.9038322567939758


In [36]:
def predict(images, model, batch_size = 64):
    images = torch.as_tensor(images, dtype= torch.float32)
    y_preds = torch.zeros((images.size(0), NUM_CLASS, images.size(2), images.size(3)), device= device)
    batch_start = 0
    batch_end = batch_size
    pbar = tqdm()
    while batch_start < images.size(0):
        image = images[batch_start : batch_end]
        with torch.inference_mode():
            image = image.to(device)
            y_pred = model(image)
            y_preds[batch_start : batch_end] = y_pred
        batch_start += batch_size
        batch_end += batch_size
        pbar.update(1)
    pbar.close()
    res = y_preds.cpu().numpy()
    del y_preds
    return res

In [ ]:
class Segmentor(pl.LightningModule):
    def __init__(self, model = PiDiNet(img_channel=1, inplane=32, dil=2, sa=False, ta=True)):
        super().__init__()
        self.model = model
    def forward(self, x):
        return self.model(x)
    def get_metrics(self):
        # don't show the version number
        items = super().get_metrics()
        items.pop("v_num", None)
        return items

    # def _step(self, batch):
    #     image, y_true = batch
    #     y_pred = self.model(image)
    #     loss = SemiActiveLoss(device=self.device)(image, y_true, y_pred)
    #     dice_score, jaccard_score = dice(y_true, y_pred), jaccard(y_true, y_pred)
    #     return loss, dice_score, jaccard_score
    def _step(self, batch):
        image, y_true = batch
        y_pred = self.model(image)
        loss = DiceLoss(device=self.device)(y_true, y_pred)
        dice_score, jaccard_score = dice(y_true, y_pred), jaccard(y_true, y_pred)
        return loss, dice_score, jaccard_score

    def training_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"loss": loss, "train_dice": dice_score, "train_jac": jaccard_score}
        self.log_dict(metrics, on_step=True, on_epoch=True, prog_bar = True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"val_loss": loss, "val_dice": dice_score, "val_jac":jaccard_score}
        self.log_dict(metrics, prog_bar = True)
        return metrics

    def test_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"test_loss": loss, "test_dice": dice_score, "test_jac":jaccard_score}
        self.log_dict(metrics, prog_bar = True)
        return metrics
        

    def configure_optimizers(self):
        optimizer = Nadam(self.parameters(), lr=1e-3)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max",
                                                         factor = 0.5, patience=16, verbose =True)
        lr_schedulers = {"scheduler": scheduler, "monitor": "val_dice"}
        return [optimizer], lr_schedulers

initialization done


In [ ]:
weight_path = sorted(glob.glob("./weights/*"), reverse=True)
weight_path

['./weights/proposed+ta_0.9525_0.9104.ckpt',
 './weights/proposed+ta_0.9173_0.8566.ckpt',
 './weights/proposed+ta_0.8944_0.8251.ckpt',
 './weights/proposed+ta_0.8429.ckpt',
 './weights/proposed+ta_0.8404.ckpt',
 './weights/proposed+sa_0.9219_0.8623.ckpt',
 './weights/proposed+sa_0.8856_0.8142.ckpt',
 './weights/propose.ckpt',
 './weights/ckpt0.9441.ckpt',
 './weights/ckpt0.9199.ckpt',
 './weights/ckpt0.9141.ckpt',
 './weights/ckpt0.9139.ckpt',
 './weights/U_Next_0.91.ckpt',
 './weights/CELoss.ckpt',
 './weights/AC.ckpt',
 './weights/AC+Local.ckpt',
 './weights/AC+Global.ckpt']

In [ ]:
x_test.shape

(30, 192, 256, 3)

In [37]:
from sklearn.metrics import recall_score, precision_score, precision_recall_fscore_support
# x_inf = np.zeros((x_test.shape[0], 3, x_test.shape[1], x_test.shape[2])) #x_inf = x_test.transpose(0, -1, 1) / 255
x_inf = x_test.transpose(0, -1, 1, 2) / 255
segmentor = Segmentor(PiDiNet(img_channel=3, inplane=32, dil=2, sa=True, ta=False)) #Segmentor(U_Net(drop_prob=0)) 
segmentor = Segmentor.load_from_checkpoint(checkpoint_path="/content/weights/ckpt0.9425.ckpt")
segmentor = segmentor.to(device)
segmentor.eval()
y_pred = predict(x_inf, segmentor)
torch.cuda.empty_cache()
mask_predict = np.argmax(y_pred, axis=1)
a = y_test.reshape(y_test.shape[0], -1)
b = mask_predict.reshape(mask_predict.shape[0], -1)
print(precision_recall_fscore_support(a, b, average="samples", zero_division=1))
inter = np.sum(a * b, axis=1)
uni = np.sum(a + b,  axis=1)
dice1 = (2*inter+1e-5) / (uni+1e-5) 

uni1 = np.sum(a + b - a*b,  axis=1)
jac = (inter+1e-5) / (uni1+1e-5)
print(np.mean(dice1), np.mean(jac))

initialization done



0it [00:00, ?it/s]
1it [00:01,  1.32s/it]


(0.9451145839051783, 0.9451531725299188, 0.9425441005464162, None)
0.9425441005873375 0.8933813858892343


In [38]:
# %cd /content/drive/MyDrive/new_paper/ISIC/dataISIC2018
np.savez_compressed("PH2_propose", predicted_mask=mask_predict)

In [ ]:
for i in range(x_test.shape[0]):
    plt.figure(i+1)
    plt.subplot(131), plt.imshow(x_test[i])
    plt.subplot(132), plt.imshow(y_test[i])
    plt.subplot(133), plt.imshow(mask_predict[i])


##Test tine augmentation

In [39]:
x_test_lr = np.flip(x_test, 2)
x_test_ud = np.flip(x_test, 1)
x_test_lr_ud = np.flip(x_test, [1,2])

In [41]:
from sklearn.metrics import recall_score, precision_score, precision_recall_fscore_support
x_pre_origin = x_test.transpose(0, -1, 1, 2) / 255
x_pre_lr = x_test_lr.transpose(0, -1, 1, 2) / 255
x_pre_ud = x_test_ud.transpose(0, -1, 1, 2) / 255
x_pre_lr_ud = x_test_lr_ud.transpose(0, -1, 1, 2) / 255
segmentor = Segmentor(PiDiNet(img_channel=3, inplane=32, dil=4, sa=True, ta=False)) #Segmentor(U_Net(drop_prob=0)) 
segmentor = Segmentor.load_from_checkpoint(checkpoint_path="/content/weights/ckpt0.9425.ckpt")
segmentor = segmentor.to(device)
segmentor.eval()
y_pred_origin = predict(x_pre_origin, segmentor)
y_pred_lr = predict(x_pre_lr, segmentor)
y_pred_ud = predict(x_pre_ud, segmentor)
y_pred_lr_ud = predict(x_pre_lr_ud, segmentor)
torch.cuda.empty_cache()

initialization done



1it [00:00, 15.92it/s]

1it [00:00, 46.52it/s]

1it [00:00, 46.65it/s]

1it [00:00, 46.13it/s]


In [42]:
y_pred_lr1 = np.flip(y_pred_lr.transpose(0, 2, 3, 1), 2)
y_pred_ud1 = np.flip(y_pred_ud.transpose(0, 2, 3, 1), 1)
y_pred_lr_ud1 = np.flip(y_pred_lr_ud.transpose(0, 2, 3, 1), [1,2])
y_pred_origin1 = y_pred_origin.transpose(0, 2, 3, 1)
y_pred = (y_pred_origin1 + y_pred_lr1 + y_pred_ud1 + y_pred_lr_ud1) / 4

In [43]:
mask_predict = np.argmax(y_pred, axis=-1)
a = y_test.reshape(y_test.shape[0], -1)
b = mask_predict.reshape(mask_predict.shape[0], -1)
print(precision_recall_fscore_support(a, b, average="samples", zero_division=1))
inter = np.sum(a * b, axis=1)
uni = np.sum(a + b,  axis=1)
dice1 = (2*inter+1e-5) / (uni+1e-5) 

uni1 = np.sum(a + b - a*b,  axis=1)
jac = (inter+1e-5) / (uni1+1e-5)
print(np.mean(dice1), np.mean(jac))
# (0.924715809366098, 0.9052908296369458, 0.9015638606435703, None)
# 0.9015638608098163 0.8357760679543831

(0.9465950466915418, 0.943827539634706, 0.9424809336907779, None)
0.9424809337315733 0.8930966171368545


In [ ]:
# for i in range(10,20):
#     plt.figure(i+1)
#     plt.subplot(131), plt.imshow(x_test_lr[i])
#     plt.subplot(132), plt.imshow(x[i])
#     plt.subplot(133), plt.imshow(x_test[i])
